###**Q1**.  Use Lucidchart to create an Entity Relationship Diagram (ERD) for the following tables representing a customer order tracking system:

Tables and fields:
1. product:
    - product_id: INTEGER (Primary key)
    - name: TEXT not null
    - price: REAL


2. customer:
    - customer_id: INTEGER (Primary key)
    - name: TEXT not null
    - email: TEXT not null


3. purchase_order:
    - order_id: INTEGER (Primary key)
    - customer_id: INTEGER (Foreign key)
    - date: TEXT not null ("YYYY-MM-DD")


4. order_item:
    - order_id: INTEGER (Foreign key)
    - product_id: INTEGER (Foreign key)
    - quantity: INTEGER
    


Export the ERD as PDF and submit it in Canvas.

###**Q2**. Create SQLite tables and load data
1. Here are the csv files for the data for the four tables:
   - product: https://raw.githubusercontent.com/csbfx/cs133/main/product.csv
   - customer: https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv
   - order_item: https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv
   - purchase_order: https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv
2. In this notebook, create the database and save it in a file called `store.db`, and create the four tables as described above.
3. Load the data in the csv files into the corresponding table.
4. Commit so that the data loaded to the tables to officially written to the tables.
5. Execute a query SELECT * from each table to make sure the data are properly loaded.
6. Execute a query using SELECT statement that queries with JOIN tables to find the purchase date, the products and quantities that a particular customer has purchased.

In [156]:
# 2.2 Create the database and save it in a file called store.db, and create the four tables as described above.
# Your code here . . .

import sqlite3
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')

gdrive='drive/MyDrive/SJSU/CS 133/'
Path(gdrive+'store.db').touch()

conn = sqlite3.connect(gdrive+'store.db') # creates a connection to the database
c = conn.cursor() # the cursor is what we'll actually use

c.execute('drop table if exists Product')
c.execute('drop table if exists Customer')
c.execute('drop table if exists Purchase_Order')
c.execute('drop table if exists Order_Item')

createProductTable = '''
CREATE TABLE IF NOT EXISTS Product
(
  product_id INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  price REAL
)
'''
createCustomerTable = '''
CREATE TABLE IF NOT EXISTS Customer
(
  customer_id INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  email TEXT NOT NULL
)
'''
createPurchaseOrderTable = '''
CREATE TABLE IF NOT EXISTS Purchase_Order
(
  order_id INTEGER PRIMARY KEY,
  customer_id INTEGER,
  date TEXT NOT NULL,
  FOREIGN KEY (customer_id) REFERENCES Customer(customer_id)
)
'''
createOrderItemTable = '''
CREATE TABLE IF NOT EXISTS Order_Item
(
  order_id INTEGER,
  product_id INTEGER,
  quantity INTEGER,
  FOREIGN KEY (order_id) REFERENCES Purchase_Order(order_id),
  FOREIGN KEY (product_id) REFERENCES Product(product_id)
)
'''

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [157]:
c.execute(createProductTable)
c.execute(createCustomerTable)
c.execute(createPurchaseOrderTable)
c.execute(createOrderItemTable)

In [158]:
# 2.3 Load the data in the csv files into the corresponding table.
# Your code here . . .

import pandas as pd
product_csv = pd.read_csv("https://raw.githubusercontent.com/csbfx/cs133/main/product.csv")
customer_csv = pd.read_csv("https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv")
order_item_csv = pd.read_csv("https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv")
purchase_order_csv = pd.read_csv("https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv")


In [159]:
product_csv

,product_id,name,price
0,0,bicycle,400
1,1,helmet,45
2,2,gloves,23
3,3,chain,48


In [160]:
customer_csv

,customer_id,name,email
0,0,Wendy Lee,wlee@bike.com
1,1,Jason Brown,jb@speed.com
2,2,Harry Potter,hp@hogwarts.edu
3,3,Godric Gryffindor,gg@hogwards.edu


In [161]:
order_item_csv

,order_id,product_id,quantity
0,0,0,1
1,0,1,1
2,1,2,2
3,1,3,1


In [162]:
purchase_order_csv

,order_id,customer_id,date
0,0,0,2020-10-19
1,1,0,2020-10-20
2,2,1,2020-10-20


In [163]:
product_csv.to_sql('Product', conn, if_exists='append', index=False)
customer_csv.to_sql('Customer', conn, if_exists='append', index=False)
order_item_csv.to_sql('Order_Item', conn, if_exists='append', index=False)
purchase_order_csv.to_sql('Purchase_Order', conn, if_exists='append', index=False)


3

In [164]:
# 2.4 Commit so that the data loaded to the tables to officially written to the tables.
# Your code here . . .

conn.commit()

In [165]:
# 2.5 Execute a query SELECT * from each table to make sure the data are properly loaded.
# Your code here . . .

c.execute("SELECT * FROM Product")
results = c.fetchall()
print(results)

c.execute("SELECT * FROM Customer")
results = c.fetchall()
print(results)

c.execute("SELECT * FROM Order_Item")
results = c.fetchall()
print(results)

c.execute("SELECT * FROM Purchase_Order")
results = c.fetchall()
print(results)

[(0, 'bicycle', 400.0), (1, 'helmet', 45.0), (2, 'gloves', 23.0), (3, 'chain', 48.0)]
[(0, 'Wendy Lee', 'wlee@bike.com'), (1, 'Jason Brown', 'jb@speed.com'), (2, 'Harry Potter', 'hp@hogwarts.edu'), (3, 'Godric Gryffindor', 'gg@hogwards.edu')]
[(0, 0, 1), (0, 1, 1), (1, 2, 2), (1, 3, 1)]
[(0, 0, '2020-10-19'), (1, 0, '2020-10-20'), (2, 1, '2020-10-20')]


In [190]:
# 2.6 Execute a query using SELECT statement that queries with JOIN tables to find the purchase date,
#    the products and quantities that a particular customer has purchased.
# Your code here . . .

c.execute(
    """
    SELECT customer.name AS customer_name, purchase_order.date, product.name AS product_name, order_item.quantity
    FROM customer
    JOIN purchase_order ON customer.customer_id = purchase_order.customer_id
    JOIN order_item ON order_item.order_id = purchase_order.order_id
    JOIN product ON product.product_id = order_item.product_id
    WHERE customer.name = 'Wendy Lee'
    """
    )
results = c.fetchall()
print(results)

[('Wendy Lee', '2020-10-19', 'bicycle', 1), ('Wendy Lee', '2020-10-19', 'helmet', 1), ('Wendy Lee', '2020-10-20', 'gloves', 2), ('Wendy Lee', '2020-10-20', 'chain', 1)]
